In [1]:
import pandas as pd

file_dir = "kcb_sdf_data/"
#file_name = "KCB_1_8000.sdf"

KCB_data_list = ['KCB_1_8000.sdf','KCB_4_32000.sdf','KCB_5_40000.sdf','KCB_6_48000.sdf',
                 'KCB_7_56000.sdf','KCB_8_64000.sdf','KCB_9_72000.sdf','KCB_10_80000.sdf',
                 'KCB_11_88000.sdf','KCB_12_96000.sdf','KCB_14_112000.sdf','KCB_15_120000.sdf',
                 'KCB_17_136000.sdf','KCB_20_157694.sdf']

output_dir = "result/"
output_file_name = "targetCompound.sdf"
final_output_file_name = "FinalResultTargetCompound.sdf"

seperator_Tag = "$$$$\n"
compound_block_num = 0
compound_block = []

final_result = ""

In [2]:
def convert_to_numeric(corp_id_str):
    return int(corp_id_str.replace(",",""))

In [3]:
import locale

def setLocale(corp_id_num):
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
    return locale.format("%d", corp_id_num, grouping=True)
    

In [4]:
screening_data_path = "screening_result_files/"

def getCorp_ids(filename):
    data_df = pd.read_csv(screening_data_path+filename)
    corp_ids = data_df["corp_id"].apply(convert_to_numeric)
    return corp_ids

target_corp_ids = getCorp_ids("screening_result.csv")

In [5]:
target_corp_ids = target_corp_ids.sort_values(ascending=True)

In [6]:
len(target_corp_ids)

139

In [7]:
def getCompoundBlockList(infile):

    compound_block_list = []
    
    for line in infile: 
        compound_block_list.append(line)
        if("<corp_id>" in line):
            corp_id = infile.readline()
            compound_block_list.append(corp_id)
        if(seperator_Tag in line):
            compound_block_list.append(corp_id)
    return compound_block_list
    
    
#infile.close()  

In [8]:
def getCompoundBlocks(compoundBlockList):
    indexes = [index for index in range(len(compoundBlockList)) if compoundBlockList[index] == seperator_Tag]
    
    compound_block_num = compoundBlockList.count(seperator_Tag)
    #print(indexes,compound_block_num)
    
    compoundBlocks = []
    for i in range(compound_block_num):
        if(i ==0):
            compoundBlock = compoundBlockList[i : indexes[i]+2]
        elif( i < compound_block_num-1):
            compoundBlock = compoundBlockList[indexes[i-1]+2 :indexes[i]+2 ]
        else:
            compoundBlock = compoundBlockList[indexes[i-1]+2:]
        compoundBlocks.append(compoundBlock)
    return compoundBlocks

In [9]:
def getTargetFile(corp_id):
    if(corp_id <= 12374 ):
        return 0
    elif(corp_id <= 44965 ):
        return 1
    elif(corp_id <= 52990 ):
        return 2
    elif(corp_id <= 64266 ):
        return 3
    elif(corp_id <= 72327 ):
        return 4
    elif(corp_id <= 88575 ):
        return 5
    elif(corp_id <= 102998 ):
        return 6
    elif(corp_id <= 111557 ):
        return 7
    elif(corp_id <= 130299 ):
        return 8
    elif(corp_id <= 140394 ):
        return 9
    elif(corp_id <= 183820 ):
        return 10
    elif(corp_id <= 197181 ):
        return 11
    elif(corp_id <= 223442 ):
        return 12
    else:
        return 13

In [38]:
'''def getTargetCompounds(block_list,target_corp_ids):
    targetCompounds = []
    for i in range(len(block_list)):
        for idx in range(len(target_corp_ids)):
            print(block_list[i][-1],target_corp_ids[idx])
            if(int(block_list[i][-1]) == target_corp_ids[idx] ):
                targetCompounds.append(block_list[i])
    return targetCompounds'''

In [10]:
def getTargetCompound(block_list,target_corp_id):
    for i in range(len(block_list)):
        #print(block_list[1])
        #print(block_list[i][-1],target_corp_id)
        if(int(block_list[i][-1]) == target_corp_id):
            #print(block_list[i][-1],target_corp_id)
            #print(block_list[i][-1])
            return block_list[i][:-1]

In [24]:
# Test 12336
'''target_corp_id = 12336
target_file_idx = getTargetFile(target_corp_id)
print(KCB_data_list[target_file_idx])
infile = open(file_dir+KCB_data_list[target_file_idx])
all_compounds_list = getCompoundBlockList(infile)
print(len(all_compounds_list))
block_list = getCompoundBlocks(all_compounds_list)
print(len(block_list))
targetCompound = getTargetCompound(block_list,target_corp_id)
#targetCompound = getTargetCompound(block_list,setLocale(target_corp_id))'''

KCB_1_8000.sdf
489618
8000


In [11]:
def generateSDF_from_list(targetCompounds, idx):
    
    with open(output_dir+idx+"_"+output_file_name, 'w') as f:
        out_data = ""
        for item in targetCompounds:
            last_item = item[-1]
            second_tag = True
            for sub_item in item:
                if last_item != sub_item:
                    f.write(str(sub_item))
                if last_item == sub_item and second_tag:
                    f.write(str(sub_item))
                    #print(str(sub_item))
                    second_tag = False
                    #print("second Tag : ",second_tag)

In [12]:
from tqdm import tqdm

for idx in tqdm(range(len(target_corp_ids))):
    target_file_idx = getTargetFile(target_corp_ids[idx])
    #print(KCB_data_list[target_file_idx])
    infile = open(file_dir+KCB_data_list[target_file_idx])
    all_compounds_list = getCompoundBlockList(infile)
    #print(len(all_compounds_list))
    block_list = getCompoundBlocks(all_compounds_list)
    #print(len(block_list))
    targetCompound = getTargetCompound(block_list,target_corp_ids[idx])
    generateSDF_from_list(targetCompound, str(idx))
    
    with open(output_dir+str(idx)+"_"+output_file_name) as fp: 
        data = fp.read() 
        final_result += data

with open (output_dir+final_output_file_name, 'w') as fp: 
    fp.write(final_result) 

100%|██████████| 139/139 [00:53<00:00,  2.60it/s]


In [28]:
targetCompound

['Molecule\n',
 '  DSViewer          3D                             0\n',
 '\n',
 ' 73 77  0  0  0  0  0  0  0  0999 V2000\n',
 '   -3.2660   -7.1561   -0.0066 C   0  0\n',
 '   -4.6574   -6.6001   -0.0018 N   0  0\n',
 '   -5.5505   -7.5262   -0.0042 C   0  0\n',
 '   -4.8029   -9.0100   -0.0118 S   0  0\n',
 '   -3.2099   -8.4957   -0.0125 C   0  0\n',
 '   -2.0387   -6.2841   -0.0036 C   0  0\n',
 '    7.9753   -5.7950    0.0547 N   0  0\n',
 '    3.0217   -5.8473    0.0084 C   0  0\n',
 '    8.1018   -4.5254    0.1157 C   0  0\n',
 '   18.3933   -0.3918    0.1981 C   0  0\n',
 '    4.2599   -6.4144    0.0014 N   0  0\n',
 '    9.4758   -3.8848    0.1449 C   0  0\n',
 '   -7.0229   -7.3203   -0.0013 C   0  0\n',
 '   12.0388   -2.6405    0.1647 C   0  0\n',
 '    1.7583   -6.7573   -0.0258 C   0  0\n',
 '   -2.1744   -5.0581    0.0064 O   0  0\n',
 '   13.4102   -1.9242    0.1343 C   0  0\n',
 '    2.8814   -4.6202    0.0472 O   0  0\n',
 '   18.3899   -1.4928    0.7583 O   0  0\n',

0